In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os
import sys
module_path = os.path.abspath(os.path.join("..", "barrels"))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import os
from pathlib import Path
import subprocess

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pycolmap
import transforms3d as t3d
import visu3d as v3d

import colmap_util as cutil

In [ ]:
image_dir = Path("../dive-data/Dive8/clips/barrelddt3")
imgpaths = sorted(list(image_dir.glob("*.jpg")))

reconstr_path = Path(f"output/{image_dir.name}-reconstr")
output_path = reconstr_path / "colmap-out"
manual_reconstr_path = output_path / "manual"
manual_reconstr_path.mkdir(exist_ok=True)
camtxt = manual_reconstr_path / "cameras.txt"
imgtxt = manual_reconstr_path / "images.txt"
pointstxt = manual_reconstr_path / "points3D.txt"
camtxt.touch()
imgtxt.touch()
pointstxt.touch()
img_navs = pd.read_csv(image_dir / "frame-time-nav.csv")
img_navs["timestamp"] = pd.to_datetime(img_navs["timestamp"])
database_path = output_path / "database.db"
images = cutil.get_images(database_path)

In [ ]:
def haversine(lat1, lat2, lon1, lon2):
    """
    Calculates the haversine distance between two points.
    """
    R = 6378.137  # Radius of earth in KM
    dLat = lat2 * np.pi / 180 - lat1 * np.pi / 180
    dLon = lon2 * np.pi / 180 - lon1 * np.pi / 180
    a = np.sin(dLat / 2) * np.sin(dLat / 2) + np.cos(lat1 * np.pi / 180) * np.cos(
        lat2 * np.pi / 180
    ) * np.sin(dLon / 2) * np.sin(dLon / 2)
    c = 2 * np.arctan2(a ** (1 / 2), (1 - a) ** (1 / 2))
    d = R * c
    return d * 1000  # meters

def get_img_navrow(filename):
    img_navrows = img_navs[img_navs["filename"] == filename]
    return img_navrows.iloc[0]

In [ ]:
with open(camtxt, "wt") as camf, open(imgtxt, "wt") as imgf:
    camf.write("# Camera list with one line of data per camera:\n")
    camf.write("#   CAMERA_ID, MODEL, WIDTH, HEIGHT, PARAMS[]\n")
    camf.write(f"# Number of cameras: {len(imgpaths)}\n")
    imgf.write("# Image list with two lines of data per image:\n")
    imgf.write("#   IMAGE_ID, QW, QX, QY, QZ, TX, TY, TZ, CAMERA_ID, NAME\n")
    imgf.write("#   POINTS2D[] as (X, Y, POINT3D_ID)\n")
    origin_row = img_navs.iloc[0]
    origin_x = origin_row["longitude"]
    origin_y = origin_row["latitude"]
    origin_z = -origin_row["depth"]
    # for idx, imgpath in enumerate(imgpaths):
    for idx, img in enumerate(images):
        imgpath = image_dir / img.name
        row = get_img_navrow(imgpath.name)
        navpos = row
        x = navpos["longitude"]
        y = navpos["latitude"]
        xdiff = haversine(origin_y, origin_y, origin_x, x)
        ydiff = haversine(origin_y, y, origin_x, origin_x)
        if x < origin_x:
            xdiff *= -1
        if y < origin_y:
            ydiff *= -1
        x = xdiff
        y = ydiff
        z = -navpos["depth"] - origin_z
        yaw = navpos["yaw"]
        # yaw = navpos["heading"] * np.pi / 180
        pitch = navpos["pitch"]
        roll = navpos["roll"]
        # pitch was written assuming regular frame, oops
        # roll is ambiguous, but is very small anyway, probably doesn't affect anything
        R0 = t3d.euler.euler2mat(0, 0, yaw - np.pi/2, axes="sxyz")
        R1 = t3d.euler.euler2mat(-pitch - (0 * np.pi / 180), 0, roll, axes="rxyz")
        R = R0 @ R1
        T = np.eye(4)
        T[:3, :3] = R
        T[:3, 3] = [x, y, z]
        # w, x, y, z
        q = t3d.quaternions.mat2quat(T[:3, :3])
        camf.write(f"{img.camera_id} RADIAL 1920 875 1246 960 420 -0.123 -0.015\n")
        imgf.write(f"{img.image_id} {q[0]} {q[1]} {q[2]} {q[3]} {x} {y} {z} {img.camera_id} {imgpath.name}\n\n")

In [ ]:
prevreconpath = manual_reconstr_path
iters = 1
for i in range(iters):
    tripath = output_path / f"manualtri{i}"
    tripath.mkdir(exist_ok=True)
    bundlepath = output_path / f"manualbundle{i}"
    bundlepath.mkdir(exist_ok=True)
    subprocess.run(["colmap", "point_triangulator", "--database_path", database_path, "--image_path", image_dir, "--input_path", prevreconpath,  "--output_path", tripath])
    subprocess.run([
        "colmap", "bundle_adjuster",
        "--input_path", tripath,
        "--output_path", bundlepath,
        "--BundleAdjustment.function_tolerance", "0.01",
    ])
    subprocess.run(["colmap", "model_converter", "--input_path", tripath, "--output_path", tripath, "--output_type", "TXT"])
    subprocess.run(["colmap", "model_converter", "--input_path", bundlepath, "--output_path", bundlepath, "--output_type", "TXT"])
    prevreconpath = bundlepath

In [ ]:
reconstruction = pycolmap.Reconstruction(output_path / "manualtri0")
print(reconstruction.summary())

In [ ]:
pts, cols = cutil.get_pc(reconstruction)
ptcloud = v3d.Point3d(p=pts, rgb=cols)
ptcloud.fig